In [ ]:
import qiskit as q, numpy as np, matplotlib.pyplot as plt, torch
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Clifford, PauliList

from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor, Resize

